# WLW.DE CRAWL

In [6]:
# 라이브러리 호출
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import random
import numpy as np
import sys #sys.exit()
from urllib import parse

# 데이터 불러오기
df = pd.read_csv("interschutz_exhibitor_1.csv")
#print(df)

# 검색 주소 설정
url_base = "https://www.wlw.de"

# 리스트 수집
n_list = []
v1_list = []
v2_list = []
v3_list = []
v4_list = []
v5_list = []


# URL으로 반복문 시작
for url_orig in df['co_name']:
    #url_orig = "Systeex Brandschutz Systeme GmbH" #검색어
    #url_orig = "Sikla GmbH"
    #url_orig = "bro u fucked"

    url_search = "/en/search?q="+parse.quote(url_orig)#.replace(" ","%20")
    #url_search = "/en/search?q=millitzer%20brandschutz%20gmbh"
    #url_search = "/en/search?q=ARASTI-Feuerschutzanlagen%20GmbH%20%26%20Co.%20KG"
    #url_search = "/en/search?q=Salzgitter%20Mannesmann%20stahlhandel%20GmbH"
    #url_search = "/en/search?q=Sikla%20GmbH"

    print("Crawling . . .",url_orig)

    counter = 0

    while True: # 될때 안될때 지멋대로라서 안되면 싹 다시 돌려버리기 (크롤링 방지 서버)
        try:
            # 1차 검색 페이지
            response = requests.get(url_base + url_search, headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"})
            html = response.text
            soup = BeautifulSoup(html,'html.parser')
            try:
                no_result = soup.select_one("#__layout > div > div:nth-child(1) > div > div > div.search-results.no-results > div > div > div > div > div.rounded-sm.bg-white.p-3.text-center.shadow-100 > h1").text.strip().split()[0]
                if no_result == "No":
                    #print("[검색결과 : 있음]")
                    print("[No Search Data]")
                    n_list.append(np.nan)
                    v1_list.append(np.nan)
                    v2_list.append(np.nan)
                    v3_list.append(np.nan)
                    v4_list.append(np.nan)
                    v5_list.append(np.nan)
                    break
            except:
                pass
                #print("[검색결과 : 없음]")
            url_deep = soup.select_one('.company-title-link').attrs['href']
            #print("1차 크롤링 성공 -",url_base + url_search)
            time.sleep(random.random()/4) # 크롤링 시간 변칙 : 0 ~ 0.25 초 사이 랜덤

            # 2차 검색 페이지
            response = requests.get(url_base + url_deep, headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"})
            html = response.text
            soup = BeautifulSoup(html,'html.parser')
            #print("[S1]")
            # 기업정보 분석 (제공 지역, 설립 년도, 직원 수)
            # 문제원인 : 4개를 가지고 있는 경우, 3개, 기업마다 다름
            # 해결방안 : 각각 Try 구문 적용, 없으면 NaN 값 적용
            t_list = []
            corp_n = soup.select_one("#business-card > div > div > div.business-card__center-box.py-2 > div.business-card__details.mb-1 > a").text.strip()
            n_list.append(corp_n)

            try:        
                t1 = soup.select_one("#facts-and-figures > div:nth-child(1) > div > div.facts-and-figures__value").text
                e1 = soup.select_one("#facts-and-figures > div:nth-child(1) > div > div.facts-and-figures__title").text
                #print("Ticker 1")
            except:
                try:
                    t1 = soup.select_one("#facts-and-figures > div.facts-and-figures__certificates > div.facts-and-figures__box > div > div.facts-and-figures__value").text
                    e1 = soup.select_one("#facts-and-figures > div.facts-and-figures__certificates > div.facts-and-figures__box > div > div.facts-and-figures__title").text
                    #print("Ticker 2")
                except:
                    t1 = np.nan
                    e1 = np.nan
                    #print("정보 1 없음!")

            try:
                t2 = soup.select_one("#facts-and-figures > div:nth-child(2) > div > div.facts-and-figures__value").text
                e2 = soup.select_one("#facts-and-figures > div:nth-child(2) > div > div.facts-and-figures__title").text
            except:
                t2 = np.nan
                e2 = np.nan
                #print("정보 2 없음!")

            try:
                t3 = soup.select_one("#facts-and-figures > div:nth-child(3) > div > div.facts-and-figures__value").text
                e3 = soup.select_one("#facts-and-figures > div:nth-child(3) > div > div.facts-and-figures__title").text
            except:
                t3 = np.nan
                e3 = np.nan
                #print("정보 3 없음!")

            try:
                t4 = soup.select_one("#facts-and-figures > div:nth-child(4) > div > div.facts-and-figures__value").text
                e4 = soup.select_one("#facts-and-figures > div:nth-child(4) > div > div.facts-and-figures__title").text
            except:
                t4 = np.nan
                e4 = np.nan
                #print("정보 4 없음!")

            # 재무정보 분석 (제공 연도, 총 자산, 매출) 크롤링 하려고 했으나
            # 자바스크립트로 작성됨, 크롤링 힘듬(동적 크롤링 필요) + 작은기업엔 없음


            # 기업분류 분석
            try:
                c1 = soup.select_one("#categories > div.categories.relative > div.categories__container > div:nth-child(1) > div > div.chip.w-full.tablet\:w-\[244px\].desktop\:w-\[344px\].flex.cursor-pointer.justify-center.items-center.py-1.px-2.text-blue-100.rounded.bg-blue-10.border-blue-50.border.relative > div.chip__title.tablet-down\:text-xl.w-full.text-ellipsis.overflow-hidden.text-lg").text.strip()
            except:
                c1 = np.nan
            
            try:
                c2 = soup.select_one("#categories > div.categories.relative > div.categories__container > div:nth-child(2) > div > div.chip.w-full.tablet\:w-\[244px\].desktop\:w-\[344px\].flex.cursor-pointer.justify-center.items-center.py-1.px-2.text-blue-100.rounded.bg-blue-10.border-blue-50.border.relative > div.chip__title.tablet-down\:text-xl.w-full.text-ellipsis.overflow-hidden.text-lg").text.strip()
            except:
                c2 = np.nan
            
            try:
                c3 = soup.select_one("#categories > div.categories.relative > div.categories__container > div:nth-child(3) > div > div.chip.w-full.tablet\:w-\[244px\].desktop\:w-\[344px\].flex.cursor-pointer.justify-center.items-center.py-1.px-2.text-blue-100.rounded.bg-blue-10.border-blue-50.border.relative > div.chip__title.tablet-down\:text-xl.w-full.text-ellipsis.overflow-hidden.text-lg").text.strip()
            except:
                c3 = np.nan

            try:    
                c4 = soup.select_one("#categories > div.categories.relative > div.categories__container > div:nth-child(4) > div > div.chip.w-full.tablet\:w-\[244px\].desktop\:w-\[344px\].flex.cursor-pointer.justify-center.items-center.py-1.px-2.text-blue-100.rounded.bg-blue-10.border-blue-50.border.relative > div.chip__title.tablet-down\:text-xl.w-full.text-ellipsis.overflow-hidden.text-lg").text.strip()
            except:
                c4 = np.nan

            try:
                c5 = soup.select_one("#categories > div.categories.relative > div.categories__container > div:nth-child(5) > div > div.chip.w-full.tablet\:w-\[244px\].desktop\:w-\[344px\].flex.cursor-pointer.justify-center.items-center.py-1.px-2.text-blue-100.rounded.bg-blue-10.border-blue-50.border.relative > div.chip__title.tablet-down\:text-xl.w-full.text-ellipsis.overflow-hidden.text-lg").text.strip()
            except:
                c5 = np.nan

            try:
                c6 = soup.select_one("#categories > div.categories.relative > div.categories__container > div:nth-child(6) > div > div.chip.w-full.tablet\:w-\[244px\].desktop\:w-\[344px\].flex.cursor-pointer.justify-center.items-center.py-1.px-2.text-blue-100.rounded.bg-blue-10.border-blue-50.border.relative > div.chip__title.tablet-down\:text-xl.w-full.text-ellipsis.overflow-hidden.text-lg").text.strip()
            except:
                c6 = np.nan

            try:
                c7 = soup.select_one("#categories > div.categories.relative > div.categories__container > div:nth-child(7) > div > div.chip.w-full.tablet\:w-\[244px\].desktop\:w-\[344px\].flex.cursor-pointer.justify-center.items-center.py-1.px-2.text-blue-100.rounded.bg-blue-10.border-blue-50.border.relative > div.chip__title.tablet-down\:text-xl.w-full.text-ellipsis.overflow-hidden.text-lg").text.strip()
            except:
                c7 = np.nan

            try:
                c8 = soup.select_one("#categories > div.categories.relative > div.categories__container > div:nth-child(7) > div > div.chip.w-full.tablet\:w-\[244px\].desktop\:w-\[344px\].flex.cursor-pointer.justify-center.items-center.py-1.px-2.text-blue-100.rounded.bg-blue-10.border-blue-50.border.relative > div.chip__title.tablet-down\:text-xl.w-full.text-ellipsis.overflow-hidden.text-lg").text.strip()
            except:
                c8 = np.nan

            c_list = []
            c_list.append(c1)
            c_list.append(c2)
            c_list.append(c3)
            c_list.append(c4)
            c_list.append(c5)
            c_list.append(c6)
            c_list.append(c7)
            c_list.append(c8)
            c_list = list(set(c_list)) #중복제거
            try:
                c_list.remove(np.nan)
            except:
                pass
                    
            #그냥 딕셔너리 구조로 Key-Value 만들자.

            
            #print("2차 크롤링 성공 -",url_base + url_deep)
            

            e_dict = {}
            e_dict[e1] = t1
            e_dict[e2] = t2
            e_dict[e3] = t3
            e_dict[e4] = t4
            
            v1_list.append(e_dict.get("Certificates")) # 없으면 None 입력됨
            v2_list.append(e_dict.get("Delivery area"))
            v3_list.append(e_dict.get("Year of foundation"))
            v4_list.append(e_dict.get("Number of employees"))
            v5_list.append(c_list)

            #print(e_dict)
            #print(c_list)
            print("[ Try :",counter+1,"] Success -",corp_n)
            break
        except:
            if counter == 9:
                print("[ Try : 10 ] Failed, F**K this S**T i'm out")
                v1_list.append(np.nan)
                v2_list.append(np.nan)
                v3_list.append(np.nan)
                v4_list.append(np.nan)
                v5_list.append(np.nan)
                break
            counter += 1
            print("[ Try :",counter,"] Failed")
            time.sleep(random.random()/4) # 크롤링 시간 변칙 : 0 ~ 0.25 초 사이 랜덤
    
if len(v1_list)==len(v2_list)==len(v3_list)==len(v4_list)==len(v5_list):
    print("[List Length] match")
else:
    print("[List length] not match")


Crawling . . . Abel & Käufl Mobilfunkhandel
[ Try : 1 ] Success - Abel & Käufl Mobilfunkhandels GmbH
Crawling . . . Abilus Group
[ Try : 1 ] Success - Abakus Group Management Ges. mbH + Co KG
Crawling . . . accellonet
[No Search Data]
Crawling . . . Access Textiles
[ Try : 1 ] Success - HOLFELDER GmbH
Crawling . . . Acco Yangin Ekipmanlari
[No Search Data]
Crawling . . . ACCURO Sonderlöschanlagen
[ Try : 1 ] Success - ACCURO Sonderlöschanlagen GmbH
Crawling . . . accu:rate
[ Try : 1 ] Success - EvoCount GmbH
Crawling . . . ace-tec
[ Try : 1 ] Success - ABE-TEC Bremen GmbH
Crawling . . . ACT special car center
[ Try : 1 ] Success - bellanet GmbH
Crawling . . . ADAC-Luftrettung
[No Search Data]
Crawling . . . ADARO Tecnologia
[No Search Data]
Crawling . . . Advancis Software & Services
[ Try : 1 ] Success - Advancis Software & Services GmbH
Crawling . . . AED ClimateBOX by Gras
[No Search Data]
Crawling . . . Aeraulique Technologie
[No Search Data]
Crawling . . . AeroClave
[ Try : 1 ] Su

In [3]:
df.columns

Index(['Unnamed: 0', 'co_name', 'sc_link', 'co_desc', 'co_loca', 'co_lono',
       'co_cont', 'co_webs', 'co_phnb'],
      dtype='object')

In [5]:
len(n_list)

814

In [7]:
df.rename(columns={'Unnamed: 0':'cp_name'}, inplace=True)
df['cp_name'] = n_list

In [8]:

df['co_cert'] = v1_list
df['co_area'] = v2_list
df['co_foyr'] = v3_list
df['co_empl'] = v4_list
df['co_catg'] = v5_list

In [16]:
# 세이브
df.to_csv('company_crawl3.csv', encoding='utf-8')

In [1]:
df

NameError: name 'df' is not defined

## Test

In [9]:
text = "abel käufl mobilfunkhandels"
temp_str = ""
for i in text:
    temp_str = temp_str+(str(ord(i)))
#print(temp_str)

from urllib import parse
print(parse.quote(text))

abel%20k%C3%A4ufl%20mobilfunkhandels


In [17]:
# 라이브러리 호출
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import random

# 검색 주소 설정
url_base = "https://www.wlw.de"
#url_search = "/en/search?q=ARASTI-Feuerschutzanlagen%20GmbH%20%26%20Co.%20KG"
url_search = "/en/search?q=millitzer%20brandschutz%20gmbh"
#url_search = "/en/search?q=Salzgitter%20Mannesmann%20stahlhandel%20GmbH"

# 1차 검색 페이지
response = requests.get(url_base + url_search,headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"})
html = response.text
soup = BeautifulSoup(html,'html.parser')
url_deep = soup.select_one('.company-title-link').attrs['href']
print(url_deep)

# 2차 검색 페이지
response = requests.get(url_base + url_deep,headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"})
html = response.text
soup = BeautifulSoup(html, "html.parser")
soup = soup.select_one("#facts-and-figures")

t1 = soup.select_one("div:nth-child(1) > div > div.facts-and-figures__value").text
t2 = soup.select_one("div:nth-child(2) > div > div.facts-and-figures__value").text
t3 = soup.select_one("div:nth-child(3) > div > div.facts-and-figures__value").text
t4 = soup.select_one("div:nth-child(4) > div > div.facts-and-figures__value").text

e1 = soup.select_one("div:nth-child(1) > div > div.facts-and-figures__title").text
e2 = soup.select_one("div:nth-child(2) > div > div.facts-and-figures__title").text
e3 = soup.select_one("div:nth-child(3) > div > div.facts-and-figures__title").text
e4 = soup.select_one("div:nth-child(4) > div > div.facts-and-figures__title").text

print(t1, t2, t3, t4)
print(e1, e2, e3, e4)

/en/company/millitzer-brandschutz-gmbh-400818
1 National 1953 50 – 99
Certificates Delivery area Year of foundation Number of employees


In [18]:
url_orig = "Systeex Brandschutz Systeme GmbH"
url_search = "/en/search?q="+url_orig.replace(" ","%20")
print(url_base + url_search)


https://www.wlw.de/en/search?q=Systeex%20Brandschutz%20Systeme%20GmbH
